# Notebook for feature processing and training the stage level probaility model




## Import reqiured libs, gcp auth and set project_id


In [ ]:
import pandas as pd
import numpy as np
import math
import warnings

warnings.filterwarnings("ignore")
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 5000

In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'buyersight'
!gcloud config set project {project_id}

Updated property [core/project].


## Download the file from GCP cloud storage and read it as dataframe


> Change the required input file path 



In [ ]:
# Download the file from a given Google Cloud Storage bucket
!gsutil cp gs://triple-draw/stage_feature_metrics/organization_13/features_13_20220303-225858933.csv /tmp/features_13_20220303-225858933.csv
  
# Read the csv file in dataframe
df_all_metrics = pd.read_csv("/tmp/features_13_20220303-225858933.csv")

Copying gs://triple-draw/stage_feature_metrics/organization_13/features_13_20220303-225858933.csv...
/ [1 files][  1.6 MiB/  1.6 MiB]                                                
Operation completed over 1 objects/1.6 MiB.                                      


## Preprocess the input metrics

In [ ]:
df_all_metrics.head()

,organization_id,account_id,opportunity_id,start_date,end_date,stage_name,stage_start_date,stage_end_date,did_progress,activity_start_date,activity_end_date,time_in_stage,total_num_of_contacts,rep_response_time_mean,prospect_response_time_mean,rep_followup_time_mean,word_count_mean,flesch_kincaid_grade_mean,daily_activity_velocity,daily_email_velocity,daily_outbound_velocity,daily_inbound_velocity,daily_call_velocity,daily_meeting_velocity,daily_sms_velocity,daily_linkedin_velocity,daily_task_velocity,starting_stage,stage_num_movements,stage_num_progress,stage_num_skipped,stage_avg_skip_size,stage_num_skipped_one,stage_num_skipped_many,stage_num_backward,stage_num_reopened,stage_num_closed,first_close_date,last_mid_stage_close_date,last_close_date,close_date_movements,close_date_push_out_count,close_date_pull_in_count,close_date_max_push_out,close_date_avg_push_out,close_date_max_pull_in,close_date_avg_pull_in,close_date_max_movement,close_date_avg_movement,close_date_final_movement,close_date_total_mid_stage_movement,close_date_final_pull_in,close_date_final_push_out,close_date_total_mid_stage_pull_in,close_date_total_mid_stage_push_out,min_close_date,max_close_date,type,is_won,amount
0,13,20545198,386023,7/23/2020,12/21/2020,SQL,7/23/2020,7/29/2020,True,7/23/2020,7/29/2020,6.144074,1,NaN,NaN,NaN,28.000000,2.500000,0.651034,0.162758,0.162758,0.000000,0.162758,0.325517,0,0.0,0.000000,SQL,0,0,0,0.0,0,0,0,0,0,9/30/2020,12/21/2020,12/21/2020,1,1,0,61,61.000000,0,0.000000,61,61.000000,82,82,0,82,0,82,11/30/2020,11/30/2020,New Business,True,0
1,13,20545198,386023,7/23/2020,12/21/2020,Proposal,7/29/2020,11/9/2020,False,7/23/2020,11/9/2020,102.823299,1,1.710776,2.831102,0.130428,95.500000,5.046677,0.192718,0.128479,0.073416,0.055062,0.027531,0.036708,0,0.0,0.018354,SQL,1,0,1,3.0,0,1,0,0,0,9/30/2020,12/21/2020,12/21/2020,4,3,1,61,51.000000,61,61.000000,61,53.500000,82,82,0,82,0,82,9/30/2020,12/31/2020,New Business,True,0
2,13,20545198,386023,7/23/2020,12/21/2020,Closed Lost,11/9/2020,11/19/2020,False,7/23/2020,11/19/2020,10.238762,1,1.591595,8.969625,25.400266,83.200000,4.527341,0.209721,0.150999,0.083888,0.067111,0.025166,0.033555,0,0.0,0.016778,SQL,2,0,1,3.0,0,1,0,0,1,9/30/2020,12/21/2020,12/21/2020,4,3,1,61,51.000000,61,61.000000,61,53.500000,82,82,0,82,0,82,9/30/2020,12/31/2020,New Business,True,0
3,13,20545198,386023,7/23/2020,12/21/2020,Proposal,11/19/2020,12/10/2020,True,7/23/2020,12/10/2020,21.163148,1,1.710776,2.831102,7.242310,76.769231,4.721162,0.391824,0.341955,0.185226,0.156729,0.021372,0.028496,0,0.0,0.014248,SQL,3,0,1,3.0,0,1,0,1,1,9/30/2020,12/21/2020,12/21/2020,6,4,2,61,41.000000,61,46.000000,61,42.666667,82,82,0,82,0,82,9/30/2020,12/31/2020,New Business,True,0
4,13,20545198,386023,7/23/2020,12/21/2020,Negotiate,12/10/2020,12/21/2020,True,7/23/2020,12/21/2020,10.701100,1,1.488500,2.543732,6.481075,71.656250,4.668330,0.436882,0.390546,0.211822,0.178725,0.019858,0.026478,0,0.0,0.013239,SQL,4,1,1,3.0,0,1,0,1,1,9/30/2020,12/21/2020,12/21/2020,9,6,3,61,29.333333,61,31.333333,61,30.000000,82,82,0,82,0,82,9/30/2020,12/31/2020,New Business,True,0


In [ ]:
# Verify data points for each stage
df_group1 = df_all_metrics.groupby(['stage_name'])['did_progress'].value_counts()
df_group1

stage_name   did_progress
Closed Lost  False            10
Closed Won   False             9
Discover     True              1
Negotiate    True            143
             False           102
Pitch        True            240
             False           181
Proposal     False           290
             True            244
Prospect     False           228
             True            183
Qualify      True              2
SQL          True            424
             False           134
Signatures   True            186
             False            59
Name: did_progress, dtype: int64

In [ ]:
# Remove unwanted stages, stage 0 is pre opportunity stage and out of scope for this project
stages_to_be_removed = ['Closed Won','Closed Lost','Stage 0', 'New Partner', 'Closed', 'Nurture', 'Trial', 'Verbal Agreement', 'Active Leads', 'Prospects', 'Proposal Sent']
df_all_metrics = df_all_metrics[~df_all_metrics["stage_name"].isin(stages_to_be_removed)]

In [ ]:
df_all_metrics.shape

(2417, 60)

In [ ]:
# Verify there should not be any nan/null in did_progress
df11 = df_all_metrics[df_all_metrics['did_progress'].isna()]
df11.shape

(0, 60)

In [ ]:
# Verify which metrics have null values and sum of it
df_all_metrics.isnull().sum()

organization_id                           0
account_id                                0
opportunity_id                            0
start_date                                0
end_date                                  0
stage_name                                0
stage_start_date                          0
stage_end_date                            0
did_progress                              0
activity_start_date                     754
activity_end_date                       754
time_in_stage                             0
total_num_of_contacts                     0
rep_response_time_mean                 1344
prospect_response_time_mean            1270
rep_followup_time_mean                 1334
word_count_mean                         795
flesch_kincaid_grade_mean               799
daily_activity_velocity                   0
daily_email_velocity                      0
daily_outbound_velocity                   0
daily_inbound_velocity                    0
daily_call_velocity             

In [ ]:
# Drop is_won(opportunity level success) and rename did_progress(stage progress success criteria)
df_all_metrics.drop(['is_won'], axis=1, inplace=True)
df_all_metrics.rename(columns={"did_progress": "is_won"}, inplace=True)

## Select stage and metrics

In [ ]:
# Select the stage
df_all_metrics = df_all_metrics[df_all_metrics['stage_name']=='Negotiate']

In [ ]:
df_metrics = df_all_metrics[[ 
               'daily_activity_velocity', 
               'daily_meeting_velocity',
               'daily_call_velocity',
               'daily_inbound_velocity',
               'daily_outbound_velocity',  

               'rep_response_time_mean',
               'prospect_response_time_mean',
               'rep_followup_time_mean',
            
               'word_count_mean', 
               'flesch_kincaid_grade_mean',	
               "total_num_of_contacts",	

               'time_in_stage', 

               'stage_avg_skip_size',
               'stage_num_backward', 
               'stage_num_reopened',
               'stage_num_skipped_many',
              
               'close_date_avg_push_out',
               'close_date_avg_pull_in',
               'close_date_push_out_count',
               'close_date_pull_in_count', 
               'close_date_total_mid_stage_pull_in',
               'close_date_total_mid_stage_push_out',
              
               'stage_name',
               'is_won']]
              #,'verbal_agreement'

In [ ]:
df_metrics.head()

,daily_activity_velocity,daily_meeting_velocity,daily_call_velocity,daily_inbound_velocity,daily_outbound_velocity,rep_response_time_mean,prospect_response_time_mean,rep_followup_time_mean,word_count_mean,flesch_kincaid_grade_mean,total_num_of_contacts,time_in_stage,stage_avg_skip_size,stage_num_backward,stage_num_reopened,stage_num_skipped_many,close_date_avg_push_out,close_date_avg_pull_in,close_date_push_out_count,close_date_pull_in_count,close_date_total_mid_stage_pull_in,close_date_total_mid_stage_push_out,stage_name,is_won
4,0.436882,0.026478,0.019858,0.178725,0.211822,1.488500,2.543732,6.481075,71.656250,4.668330,1,10.701100,3.0,0,1,1,29.333333,31.333333,6,3,0,82,Negotiate,True
11,0.323862,0.011168,0.061422,0.033503,0.189850,2.228137,0.837685,25.731281,112.617647,5.064550,7,15.044109,2.0,0,0,0,46.000000,0.000000,2,0,0,92,Negotiate,False
19,0.168752,0.013323,0.000000,0.075494,0.071054,1.026611,3.646943,2.021871,51.437500,5.669231,2,0.003681,2.0,0,0,0,61.000000,0.000000,4,0,0,262,Negotiate,False
21,0.281614,0.022149,0.009493,0.113911,0.129732,1.026611,3.646943,2.355764,48.146341,4.820760,5,61.937257,2.0,1,0,0,52.400000,0.000000,5,0,0,262,Negotiate,True
25,0.203218,0.000000,0.064174,0.026739,0.112305,8.693319,0.823047,4.162533,54.000000,3.605186,1,14.093507,2.0,1,0,0,48.400000,61.000000,5,1,0,181,Negotiate,False


## Impute missing values

In [ ]:
metrics_for_imputer = [ 'daily_activity_velocity', 
                        'daily_meeting_velocity',
                        'daily_call_velocity',
                        'daily_inbound_velocity',
                        'daily_outbound_velocity',  

                        'rep_response_time_mean',
                        'prospect_response_time_mean',
                        'rep_followup_time_mean',
                      
                        'word_count_mean', 
                        'flesch_kincaid_grade_mean',	
                        "total_num_of_contacts",	

                        'time_in_stage']

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer = imputer.fit(df_metrics.loc[:,metrics_for_imputer])

df_metrics.loc[:,metrics_for_imputer] = imputer.transform(df_metrics.loc[:,metrics_for_imputer])
#df_metrics.loc[:,metrics_for_imputer]

In [ ]:
# Save imputer pickle
import pickle 

# Provide the GCP filepath where imputer needs to be saved
filename = "/tmp/new_Negotiation_8.pickle"
pickle.dump(imputer, open(filename, 'wb'))

# Save imputer model to GCP
!gsutil cp /tmp/new_Negotiation_8.pickle gs://triple-draw/stage_level_probability_model/model_artifacts/imputers/Negotiation_model.joblib

Copying file:///tmp/new_Negotiation_8.pickle [Content-Type=application/octet-stream]...
/ [1 files][  962.0 B/  962.0 B]                                                
Operation completed over 1 objects/962.0 B.                                      


In [ ]:
# Load imputer and test
# np.set_printoptions(suppress=True)
# loaded_imputer = pickle.load(open(filename, 'rb'))
# result = loaded_imputer.transform([[np.nan,np.nan,0.019858,0.178725,np.nan,1.488500,2.543732,np.nan,np.nan,np.nan,np.nan,np.nan]])
# print(result)

In [ ]:
df_metrics.shape

(245, 24)

In [ ]:
# Verify which metrics have null values and sum of it
df_metrics.isnull().sum()

daily_activity_velocity                0
daily_meeting_velocity                 0
daily_call_velocity                    0
daily_inbound_velocity                 0
daily_outbound_velocity                0
rep_response_time_mean                 0
prospect_response_time_mean            0
rep_followup_time_mean                 0
word_count_mean                        0
flesch_kincaid_grade_mean              0
total_num_of_contacts                  0
time_in_stage                          0
stage_avg_skip_size                    0
stage_num_backward                     0
stage_num_reopened                     0
stage_num_skipped_many                 0
close_date_avg_push_out                0
close_date_avg_pull_in                 0
close_date_push_out_count              0
close_date_pull_in_count               0
close_date_total_mid_stage_pull_in     0
close_date_total_mid_stage_push_out    0
stage_name                             0
is_won                                 0
dtype: int64

In [ ]:
# Drop rows if null in did progress
df_metrics.drop(df_metrics[(df_metrics['is_won'].isnull())].index, inplace=True)

## Feature Engineering

In [ ]:
# This method is used to get the log of given metrics
# As math.log(1) = 0, log(x+1) is used

def apply_log(list_of_metrics, df):
  for metric in list_of_metrics:
    df[metric] = df[metric].apply(lambda x: math.log(x+1))
  
  return df

In [ ]:
metrics_for_log = ['total_num_of_contacts', 'time_in_stage', 'word_count_mean',
                   'rep_response_time_mean', 'prospect_response_time_mean', 'rep_followup_time_mean',
                   
                   'daily_activity_velocity', 'daily_outbound_velocity', 'daily_inbound_velocity',
                   'daily_call_velocity', 'daily_meeting_velocity', 
                   
                   'close_date_avg_push_out', 'close_date_avg_pull_in',
                   'close_date_total_mid_stage_push_out', 'close_date_total_mid_stage_pull_in']

# Apply log function to get the log of metrics
df_final = apply_log(metrics_for_log, df_metrics)

In [ ]:
df_final.head()

,daily_activity_velocity,daily_meeting_velocity,daily_call_velocity,daily_inbound_velocity,daily_outbound_velocity,rep_response_time_mean,prospect_response_time_mean,rep_followup_time_mean,word_count_mean,flesch_kincaid_grade_mean,total_num_of_contacts,time_in_stage,stage_avg_skip_size,stage_num_backward,stage_num_reopened,stage_num_skipped_many,close_date_avg_push_out,close_date_avg_pull_in,close_date_push_out_count,close_date_pull_in_count,close_date_total_mid_stage_pull_in,close_date_total_mid_stage_push_out,stage_name,is_won
4,0.362476,0.026133,0.019664,0.164433,0.192125,0.911680,1.265181,2.012376,4.285739,4.668330,0.693147,2.459683,3.0,0,1,1,3.412247,3.476099,6,3,0.0,4.418841,Negotiate,True
11,0.280554,0.011106,0.059610,0.032954,0.173828,1.171905,0.608507,3.285834,4.732839,5.064550,2.079442,2.775342,2.0,0,0,0,3.850148,0.000000,2,0,0.0,4.532599,Negotiate,False
19,0.155937,0.013235,0.000000,0.072780,0.068643,0.706365,1.536210,1.105876,3.959622,5.669231,1.098612,0.003674,2.0,0,0,0,4.127134,0.000000,4,0,0.0,5.572154,Negotiate,False
21,0.248120,0.021908,0.009448,0.107877,0.121981,0.706365,1.536210,1.210679,3.894802,4.820760,1.791759,4.142138,2.0,1,0,0,3.977811,0.000000,5,0,0.0,5.572154,Negotiate,True
25,0.184999,0.000000,0.062199,0.026388,0.106434,2.271437,0.600509,1.641427,4.007333,3.605186,0.693147,2.714265,2.0,1,0,0,3.899950,4.127134,5,1,0.0,5.204007,Negotiate,False


In [ ]:
# Conver is_won to bool
df_final["is_won"] = df_final["is_won"].astype(bool)

## Splitting the dataset into the Training set and Test set


In [ ]:
from sklearn.model_selection import train_test_split
np.set_printoptions(precision=3, threshold=np.inf)

X = df_final.drop(['is_won', 'stage_name'],axis=1).values

y = df_final['is_won'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=None, shuffle=True)

## Normalize the data

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test =  scaler.transform(X_test)

## SVM


In [ ]:
from scipy.stats.stats import gmean
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC

classifier_svm = SVC(C=10, gamma=1)
classifier = CalibratedClassifierCV(base_estimator=classifier_svm, cv=3)

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[19,  9],
       [10, 36]])

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='macro')

0.7289377289377289

In [ ]:
print(classifier.classes_)

[False  True]


In [ ]:
y_pred_proba = classifier.predict_proba(X_test)
y_pred_proba = np.argmax(y_pred_proba, axis=1)
y_pred_proba

array([1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0])

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_pred_proba, y_pred, average='macro')

1.0

## Grid

In [ ]:
from scipy.stats.stats import gmean
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

svm_est = Pipeline([('svc',SVC(probability=True))])
Cs = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
gammas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

param_grid = dict(svc__gamma=gammas, svc__C=Cs)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=7)


grid_cv = GridSearchCV(svm_est, param_grid=param_grid, cv=cv, scoring="f1_macro")
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=7, test_size=0.2,
            train_size=None),
             estimator=Pipeline(steps=[('svc', SVC(probability=True))]),
             param_grid={'svc__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='f1_macro')

In [ ]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'svc__C': 1, 'svc__gamma': 0.1}
0.7499161053999763


## Save model


> Change the filepath to where model needs to be saved in gcp


In [ ]:
# Save probability model pickle file
import joblib

filename ='/tmp/Negotiation_model.joblib'
joblib.dump(classifier, filename)

!gsutil cp /tmp/Negotiation_model.joblib gs://triple-draw/stage_level_probability_model/model_artifacts/org_13/Negotiate_13/Negotiation_model.joblib

Copying file:///tmp/Negotiation_model.joblib [Content-Type=application/octet-stream]...
/ [1 files][ 49.1 KiB/ 49.1 KiB]                                                
Operation completed over 1 objects/49.1 KiB.                                     


In [ ]:
# Verify the model

#loaded_model = joblib.load(filename)
#loaded_model.predict_proba([[0.21114538798804383,0.0,0.0,0.14457461017319456,0.24502126974613753,0.015558234340554493,1.0,0.9999999999999999,0.0,0.38367447273686067,0.2789429456511298,0.7733581292498118,0.5,0.6666666666666666,0.0,0.0,0.8720575436985503,1.0,1.0,0.25,0.0,1.0]])
#loaded_model.predict_proba([[0.141,0,1,0,0.306,1,0,0.246,0.134,0,0,0.655,0,1,0,0,0.682,1,0.75,0.333,0,0.681]])